In [10]:
import pandas as pd


In [11]:
#read all data
orders_header = pd.read_csv('../../data/order_header.csv')
orders_details = pd.read_csv('../../data/order_details.csv')
products = pd.read_csv('../../data/products.csv')

payment_methods = pd.read_csv('../../data/payment_methods.csv')
branches = pd.read_csv('../../data/branches.csv')
categories = pd.read_csv('../../data/categories.csv')






# Orders Headers

In [12]:
orders_header.head()

,order_id,branch_id,order_date,order_total_price,order_source,order_type,order_status
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 14:03:36+03:00,15.00,1,1,4
1,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 13:41:05+03:00,6.80,1,1,4
2,3c811baa-93bc-4200-9204-fdee8932f020,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 13:39:04+03:00,5.00,1,1,4
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,975b3d24-cb71-4df8-930e-054bcd67f90c,2022-10-02 16:36:54+03:00,20.75,1,2,4
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,975b3d24-cb71-4df8-930e-054bcd67f90c,2022-10-02 16:15:22+03:00,30.00,1,1,4


In [13]:
orders_types = {1:'Dine In', 2:'Pick Up', 3:'Delivery', '4':'Drive Thru'}
orders_sources = {1:'Cashier', 2:'API', 3:'Call Center'}
orders_statuses = {1:'Pending', 2:'Active', 3:'Declined', 4:'Closed', 5:'Returned', 6:'Joined', 7:'Void'}
# orders_delivery_statuses = {1:'sent to kitchen', 2:'ready', 3:'assigned', 4:'en route', 5:'delivered', 6:'closed'}
# products_statuses = {1:'Pending', 2:'Active', 3:'Closed', 4:'Moved', 5:'Void', 6:'Returned', 7:'Declined'}
# discounts_types = {1:'Open', 2:'Predefined', 3:'Coupon', 4:'Loyalty', 5:'Promotion'}

#---------------

orders_header['order_type'] = orders_header['order_type'].map(orders_types)
orders_header['order_source'] = orders_header['order_source'].map(orders_sources)
orders_header['order_status'] = orders_header['order_status'].map(orders_statuses)


In [14]:
orders_header['order_datetime'] = pd.to_datetime(orders_header['order_date'])

# orders_header['order_time'] = orders_header['order_datetime'].dt.time
# orders_header['order_date'] = orders_header['order_datetime'].dt.date

# orders_header['hour'] = orders_header['order_datetime'].dt.hour
# orders_header['day_name'] = orders_header['order_datetime'].dt.day_name()
# orders_header['is_weekend'] = orders_header['order_datetime'].dt.dayofweek.isin([4, 5])
# orders_header['is_weekend'].replace({True:'Yes', False:'No'}, inplace=True)
# orders_header['month'] = orders_header['order_datetime'].dt.month
# orders_header['year'] = orders_header['order_datetime'].dt.year




In [15]:
orders_header.drop_duplicates(inplace=True)

In [16]:
orders_header.isnull().sum()

order_id             0
branch_id            0
order_date           0
order_total_price    0
order_source         0
order_type           0
order_status         4
order_datetime       0
dtype: int64

In [17]:
orders_header.head()

,order_id,branch_id,order_date,order_total_price,order_source,order_type,order_status,order_datetime
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 14:03:36+03:00,15.00,Cashier,Dine In,Closed,2022-10-02 14:03:36+03:00
1,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 13:41:05+03:00,6.80,Cashier,Dine In,Closed,2022-10-02 13:41:05+03:00
2,3c811baa-93bc-4200-9204-fdee8932f020,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 13:39:04+03:00,5.00,Cashier,Dine In,Closed,2022-10-02 13:39:04+03:00
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,975b3d24-cb71-4df8-930e-054bcd67f90c,2022-10-02 16:36:54+03:00,20.75,Cashier,Pick Up,Closed,2022-10-02 16:36:54+03:00
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,975b3d24-cb71-4df8-930e-054bcd67f90c,2022-10-02 16:15:22+03:00,30.00,Cashier,Dine In,Closed,2022-10-02 16:15:22+03:00


In [18]:
orders_header.rename(columns={'order_id':'order_header_id'}, inplace=True)

In [19]:
orders_header['order_status'].fillna('Void', inplace=True)

In [20]:
orders_header.rename(columns={'order_id':'order_header_id'}, inplace=True)

In [21]:
orders_header.to_csv('../../data/processed/order_header.csv', index=False)

# Order Details

In [22]:
orders_details.head()

,order_id,branch_id,order_date,order_total_price,order_source,order_type,order_status,product_id,category_id,quantity,price
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 14:03:36+03:00,15.0,1,1,4,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
1,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 14:03:36+03:00,15.0,1,1,4,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.5
2,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 14:03:36+03:00,15.0,1,1,4,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.0
3,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 13:41:05+03:00,6.8,1,1,4,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
4,5babceee-5385-4fa5-b25f-4f1638a70696,975b3d24-ce3d-4801-9c11-582a817cc591,2022-10-02 13:41:05+03:00,6.8,1,1,4,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,0.3


In [23]:
filtered_orders_details = orders_details.drop(columns=['order_date', 'order_total_price','order_source', 'order_type', 'order_status', 'branch_id'])
filtered_orders_details.head()


,order_id,product_id,category_id,quantity,price
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
1,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.5
2,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.0
3,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5
4,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,0.3


In [24]:
#filtered_orders_details['order_line_id'] = filtered_orders_details['order_id'] + '-' + filtered_orders_details['product_id'].apply(lambda x: x.split('-')[1]) + 
import uuid
filtered_orders_details['order_details_id'] = [str(uuid.uuid4()) for _ in range(len(filtered_orders_details))]
filtered_orders_details.rename(columns={'order_id':'order_header_id'}, inplace=True)

In [25]:
filtered_orders_details.head()

,order_header_id,product_id,category_id,quantity,price,order_details_id
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5,1f60caa6-c2da-4331-a281-255a07b2bd02
1,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.5,2cfe3107-421c-4527-a193-a9dc668a77f0
2,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,5.0,c5084ce0-d3a7-460e-bd5d-4459a838da12
3,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,4.5,ebbad82a-bd75-46f9-9422-f7335d8c4cd4
4,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,0.3,18700782-a80f-4713-b665-7c3084ad4a31


In [26]:
filtered_orders_details.drop_duplicates(inplace=True)

In [27]:
filtered_orders_details.to_csv('../../data/processed/order_details.csv', index=False)

# Products

In [28]:
products.head()

,product_id,product_sku,product_name,category_name,category_id,is_active,is_stock_product,price
0,975b583e-8e4e-4500-b416-e77dd352726a,psk-14,تشكس فرايز زنجر,Fries,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,5.00
1,975b583e-90a9-420a-804e-b41dddbe0971,psk-5,كلاسيك,Shawarma,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,1.50
2,975b583e-90af-4319-87d3-022d8f1936d4,psk-13,تشكس ستريبس,Chicken burger,975b5737-ccc4-4c1f-8599-5e246eee4a3b,True,False,4.50
3,975b583e-9145-4e05-90e7-9fa596c38888,psk-16,فرايز باجبنة,Fries,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,2.50
4,975b583e-91e6-424e-851d-4de6a37813f5,psk-9,اربيان دجاج l,Shawarma,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,3.75


In [29]:
products.head()

,product_id,product_sku,product_name,category_name,category_id,is_active,is_stock_product,price
0,975b583e-8e4e-4500-b416-e77dd352726a,psk-14,تشكس فرايز زنجر,Fries,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,5.00
1,975b583e-90a9-420a-804e-b41dddbe0971,psk-5,كلاسيك,Shawarma,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,1.50
2,975b583e-90af-4319-87d3-022d8f1936d4,psk-13,تشكس ستريبس,Chicken burger,975b5737-ccc4-4c1f-8599-5e246eee4a3b,True,False,4.50
3,975b583e-9145-4e05-90e7-9fa596c38888,psk-16,فرايز باجبنة,Fries,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,2.50
4,975b583e-91e6-424e-851d-4de6a37813f5,psk-9,اربيان دجاج l,Shawarma,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,3.75


In [30]:
products.product_name.unique()

array(['تشكس فرايز زنجر', 'كلاسيك', 'تشكس ستريبس', 'فرايز باجبنة',
       'اربيان دجاج l', 'تشكس برجر', 'تشيزي', 'اوريجنال', 'بيج ميل',
       'فرايز', 'ديبي برجر', 'تشكس فرايز شاورما', 'موهيتو', 'كوكتيل',
       'برو', 'هوت دجاج', 'ذا بارتي', 'بيبسي', 'اربيان دجاج xl',
       'شيك دجاج', 'مياه', 'سفن', 'هوت صوص', 'مرندا', 'بيبسي دايت', 'ثوم',
       'سفن دايت', 'مخلل', 'جبنة', 'تشكس صوص', 'رانش', 'دبس رمان',
       'راب شيكس', 'صوص ثوم حار', 'فرشلي', '5 اوريجنال عرض كريم 5JD',
       'عرض بيج شيكس / عائلية  / 10JOD', 'عرض 2 وجبة لاج دجاج 5 د',
       'وجبة مشاركة شاورما +زنجر + 2 بيبسي // كريم+ طلبات',
       'وجبة مشاركة برغر + زنجر + 2 بيبسي // كريم+ طلبات', 'وجبة موظفين',
       'عصير موسمي', 'وجبة لارج', 'بيج شيكس عرض فوديز 9 د',
       'وجبة مشاركة برجر عرض 10 د', 'عرض وجبة عائلية 10د',
       'عرض وجبة الاصدقاء  / مشاركة شاورما 6.500 JOD',
       'عرض كريم 2 وجبة لارج 5 د', 'عرض كريم XL اربيان 50%',
       'عرض كريم 2 اورجينال+1 بطاط +1 بيبسي',
       'شيكس برغر / طلبات /  بيبسي

In [31]:
products.product_sku.unique()

array(['psk-14', 'psk-5', 'psk-13', 'psk-16', 'psk-9', 'psk-11', 'psk-6',
       'psk-4', 'psk-8', 'psk-17', 'psk-12', 'psk-15', 'psk-20', 'psk-18',
       'psk-3', 'psk-2', 'psk-19', 'psk-21', 'psk-10', 'psk-7', 'psk-26',
       'psk-23', 'psk-30', 'psk-25', 'psk-22', 'psk-29', 'psk-24',
       'psk-28', 'psk-32', 'psk-27', 'psk-31', 'psk-33', 'sk-0106',
       'sk-0107', 'sk-0108', 'sk-0109', 'sk-0110', 'sk-0111', 'sk-0112',
       'sk-0113', 'sk-0114', 'sk-0115', 'sk-0116', 'sk-0117', 'sk-0118',
       'sk-0119', 'sk-0120', 'sk-0121', 'sk-0122', 'sk-0123', 'sk-0124',
       'sk-0125', 'sk-0126', 'sk-0128', 'sk-0129', 'sk-0130'],
      dtype=object)

In [32]:
products.product_sku.nunique()

56

In [33]:
products.to_csv('../../data/processed/products.csv', index=False)

# Branches

In [34]:
branches.head()

,branch_id,branch_name,opening_from,opening_to
0,975b3d24-cb71-4df8-930e-054bcd67f90c,4Chicks Abdoun,09:00,03:00
1,975b3d24-ce3d-4801-9c11-582a817cc591,4Chicks 7th circle,10:00,03:00
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,4Chicks Al-Jubeha,10:00,03:00


In [35]:
branches['opening_from'] = pd.to_datetime(branches['opening_from'])
branches['opening_to'] = pd.to_datetime(branches['opening_to'])

branches['opening_from'] = branches['opening_from'].dt.hour
branches['opening_to'] = branches['opening_to'].dt.hour

In [36]:
branches.to_csv('../../data/processed/branches.csv', index=False)

# Categories

In [37]:
categories

,category_id,category_name
0,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,Shawarma
1,975b5737-c982-4e3f-9faf-083ca9dc56ac,Fries
2,975b5737-cad1-4031-9319-5e3a7addba42,Drinks
3,975b5737-ccc4-4c1f-8599-5e246eee4a3b,Chicken burger
4,975b5737-cf8c-4ef3-a445-7c4a71f349c5,Sauces
5,975b5737-d227-4b7c-9962-3a7b09103597,Juices


In [38]:
categories.to_csv('../../data/processed/categories.csv', index=False)

In [39]:
# def check_values_exist(list1, list2):
#     return all(value in list2 for value in list1)

# check_values_exist(products.category_id.unique(), categories.category_id.unique())